# Create synthetic samples from UNSW dataset

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datacompy
import os, sys
import numpy as np
import re
import ast
import gower
from pathlib import Path
import openpyxl
import itertools

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV,
    RandomizedSearchCV
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score, accuracy_score, pairwise_distances, make_scorer, precision_score, f1_score, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy import stats
from scipy.stats import chi2_contingency, f_oneway, friedmanchisquare, wilcoxon
from scipy.spatial import distance
from joblib import dump, load

# modele
from xgboost import XGBClassifier, XGBRFClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# methods
from imblearn.under_sampling import ClusterCentroids, NearMiss
from scipy.optimize import differential_evolution
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, HDBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier, NearestNeighbors
from sklearn.inspection import permutation_importance
from scipy.spatial.distance import euclidean
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
#from ctgan import CTGAN

C:\Users\mateu\AppData\Local\Temp\ipykernel_20668\1422509104.py:5: UserWarning: SparkPandasCompare currently only supports Numpy < 2.Please note that the SparkPandasCompare functionality will not work and currently is not supported.
  import datacompy


In [3]:
#df_unsw_features = pd.read_csv('D:\\ml\\undersampling_data\\data\\unsw\\NUSW-NB15_features.csv')
df_unsw_1 = pd.read_csv('D:\\ml\\undersampling_data\\data\\unsw\\UNSW-NB15_1.csv')
df_unsw_2 = pd.read_csv('D:\\ml\\undersampling_data\\data\\unsw\\UNSW-NB15_2.csv')
df_unsw_3 = pd.read_csv('D:\\ml\\undersampling_data\\data\\unsw\\UNSW-NB15_3.csv')
df_unsw_4 = pd.read_csv('D:\\ml\\undersampling_data\\data\\unsw\\UNSW-NB15_4.csv')

C:\Users\mateu\AppData\Local\Temp\ipykernel_20668\3554131459.py:2: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df_unsw_1 = pd.read_csv('D:\\ml\\undersampling_data\\data\\unsw\\UNSW-NB15_1.csv')
C:\Users\mateu\AppData\Local\Temp\ipykernel_20668\3554131459.py:3: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df_unsw_2 = pd.read_csv('D:\\ml\\undersampling_data\\data\\unsw\\UNSW-NB15_2.csv')


In [4]:
df_unsw = pd.concat([df_unsw_1, df_unsw_2, df_unsw_3, df_unsw_4], ignore_index=True)
df_unsw = df_unsw.drop(columns=['attack_cat'])
df_unsw.head()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0.0,0,3,7,1,3,1,1,1,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0.0,0,2,4,2,3,1,1,2,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0.0,0,12,8,1,2,2,1,1,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0.0,0,6,9,1,1,1,1,1,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0.0,0,7,9,1,1,1,1,1,0


In [5]:
# what to do with ports? src drop because is random, dst - keep because is related to service, port number >1000 is random
def to_int_or_hex(x):
    try:
        # jeśli to już liczba — zwróć bez zmian
        if isinstance(x, (int, float)) and not pd.isna(x):
            return int(x)
        # jeśli tekst zaczyna się od 0x — zamień z systemu 16
        elif isinstance(x, str) and x.startswith('0x'):
            return int(x, 16)
        # w pozostałych przypadkach — spróbuj jako int z tekstu
        elif isinstance(x, str):
            return int(x)
        else:
            return pd.NA
    except ValueError:
        return pd.NA

# przykład: konwersja kolumny dsport
df_unsw['dsport'] = df_unsw['dsport'].apply(to_int_or_hex).astype('Int64')

mask_well_known = df_unsw['dsport'] <= 1000
df_well_known = df_unsw[mask_well_known].copy()
df_other_ports = df_unsw[~mask_well_known].copy()

# 2️⃣ One-Hot Encoding dla portów ≤ 1000
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_ports = encoder.fit_transform(df_well_known[['dsport']])
encoded_df = pd.DataFrame(
    encoded_ports,
    columns=encoder.get_feature_names_out(['dsport']),
    index=df_well_known.index
)

# 3️⃣ Połącz zakodowane porty z resztą danych
df_well_known = pd.concat([encoded_df, df_well_known.drop(columns=['dsport'])], axis=1)

# 4️⃣ Dla portów > 1000 utwórz kolumnę 'dsport_other'
df_other_ports['dsport_other'] = 1

# 5️⃣ Wyrównaj kolumny, żeby scalenie się udało
for col in df_well_known.columns:
    if col not in df_other_ports.columns:
        df_other_ports[col] = 0

# 6️⃣ Połącz z powrotem obie części
df_unsw = pd.concat([df_well_known, df_other_ports], axis=0).sort_index().reset_index(drop=True)

C:\Users\mateu\AppData\Local\Temp\ipykernel_20668\2688879982.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_other_ports[col] = 0
C:\Users\mateu\AppData\Local\Temp\ipykernel_20668\2688879982.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_other_ports[col] = 0
C:\Users\mateu\AppData\Local\Temp\ipykernel_20668\2688879982.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using 

In [ ]:
# insert 'dsport_other' next to 'proto' and fill NaN with 0
col = df_unsw.pop('dsport_other')
dst_col = df_unsw.columns.get_loc('proto')
df_unsw.insert(dst_col, 'dsport_other', col)
df_unsw['dsport_other'] = df_unsw['dsport_other'].fillna(0)
df_unsw = df_unsw.drop(columns=['dsport', 'srcip', 'dstip', 'sport'])

In [9]:

df_unsw.head()

,dsport_0.0,dsport_10.0,dsport_21.0,dsport_22.0,dsport_23.0,dsport_25.0,dsport_31.0,dsport_32.0,dsport_37.0,dsport_42.0,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,3,7,1,3,1,1,1,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,2,4,2,3,1,1,2,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,12,8,1,2,2,1,1,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,6,9,1,1,1,1,1,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,7,9,1,1,1,1,1,0


In [ ]:

df_unsw['proto_simplified'] = df_unsw['proto'].apply(
    lambda x: x if x in ['tcp', 'udp'] else 'other'
)

df_unsw = pd.get_dummies(df_unsw, columns=['proto_simplified'], prefix='proto')
#df_unsw = df_unsw.loc[:, ~df_unsw.columns.duplicated(keep='first')]

col = df_unsw.pop('proto_other')
col2 = df_unsw.pop('proto_tcp')
col3 = df_unsw.pop('proto_udp')
dst_col = df_unsw.columns.get_loc('state')
df_unsw.insert(dst_col, 'proto_other', col)
df_unsw.insert(dst_col + 1, 'proto_tcp', col2)
df_unsw.insert(dst_col + 2, 'proto_udp', col3)

df_unsw = df_unsw.drop(columns=['proto'])

df_unsw.loc[:, df_unsw.columns.str.startswith('proto')] = (
    df_unsw.loc[:, df_unsw.columns.str.startswith('proto')].astype(int)
)

" proto_index = df_unsw.columns.get_loc('proto')\n\nfor i, col in enumerate(df_proto.columns):\n    df_unsw.insert(proto_index + 1 + i, col, df_unsw[col]) "

In [22]:
df_unsw['state'].value_counts()

state
FIN    1478689
CON     560588
INT     490471
REQ       9043
RST        528
ECO        337
CLO        161
URH        108
ACC         43
PAR         28
ECR          9
TST          8
URN          7
no           7
MAS          7
TXD          6
Name: count, dtype: int64

In [23]:
df_unsw['service'].value_counts()

service
-           1246390
dns          781668
http         206273
ftp-data     125783
smtp          81645
ftp           49090
ssh           47160
pop3           1533
dhcp            172
ssl             142
snmp            113
radius           40
irc              31
Name: count, dtype: int64

In [21]:
df_unsw.head()

,dsport_0.0,dsport_10.0,dsport_21.0,dsport_22.0,dsport_23.0,dsport_25.0,dsport_31.0,dsport_32.0,dsport_37.0,dsport_42.0,dsport_50.0,dsport_53.0,dsport_67.0,dsport_68.0,dsport_69.0,dsport_79.0,dsport_80.0,dsport_81.0,dsport_87.0,dsport_88.0,dsport_89.0,dsport_91.0,dsport_98.0,dsport_102.0,dsport_103.0,dsport_105.0,dsport_110.0,dsport_111.0,dsport_119.0,dsport_123.0,dsport_125.0,dsport_135.0,dsport_137.0,dsport_138.0,dsport_139.0,dsport_141.0,dsport_143.0,dsport_161.0,dsport_162.0,dsport_166.0,dsport_167.0,dsport_179.0,dsport_186.0,dsport_237.0,dsport_257.0,dsport_288.0,dsport_362.0,dsport_368.0,dsport_379.0,dsport_383.0,...,dsport_993.0,dsport_997.0,dsport_998.0,dsport_other,proto_other,proto_tcp,proto_udp,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,1,CON,0.001055,132,164,31,29,0,0,dns,500473.93750,621800.93750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0.0,0.0,0,3,7,1,3,1,1,1,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0,0,1,CON,0.036133,528,304,31,29,0,0,-,87676.08594,50480.17188,4,4,0,0,0,0,132,76,0,0,9.89101,10.682733,1421927414,1421927414,7.005,7.564333,0.0,0.0,0.0,0,0,0.0,0.0,0,2,4,2,3,1,1,2,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,1,CON,0.001119,146,178,31,29,0,0,dns,521894.53130,636282.37500,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0.0,0.0,0,12,8,1,2,2,1,1,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,1,CON,0.001209,132,164,31,29,0,0,dns,436724.56250,542597.18750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.043,0.014000,0.0,0.0,0.0,0,0,0.0,0.0,0,6,9,1,1,1,1,1,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,1,CON,0.001169,146,178,31,29,0,0,dns,499572.25000,609067.56250,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.005,0.003000,0.0,0.0,0.0,0,0,0.0,0.0,0,7,9,1,1,1,1,1,0


In [24]:
df_unsw.to_csv('D:\\ml\\undersampling_data\\data\\unsw\\UNSW-NB15_processed2.csv', index=False)

In [ ]:
encoded_ports = encoder.fit_transform(df_unsw[['proto']])
encoded_df = pd.DataFrame(
    encoded_ports,
    columns=encoder.get_feature_names_out(['proto']),
    index=df_unsw.index
)
proto_index = df_unsw.columns.get_loc('proto')

for i, col in enumerate(encoded_df.columns):
    df_unsw.insert(proto_index + 1 + i, col, encoded_df[col])

In [ ]:
# reduced protocols to tcp, udp, other
df_unsw['proto'].value_counts()

In [ ]:
df_unsw = pd.get_dummies(df_unsw, columns=['proto'], prefix='proto')